In [13]:
import torch  
from torch.utils.data import Dataset, DataLoader  
import torchvision.transforms as transforms  
from torchvision.datasets import ImageFolder  
from PIL import Image  
import pathlib  
import cupy as cp 
from torchvision.datasets import ImageFolder
from torchvision import datasets
import torch  
import torch.nn as nn  
import torch.optim as optim  

In [14]:
# Verificar la disponibilidad de la GPU  
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 
print("GPU disponible:", torch.cuda.is_available())
print("Nombre de la GPU:", torch.cuda.get_device_name() if torch.cuda.is_available() else "N/A")


GPU disponible: True
Nombre de la GPU: NVIDIA GeForce RTX 4060


In [15]:
# Definir directorios
directorio_dataset = '../Datasets/CIFAR-10 dos clases'
imagenes_originales = '../Datasets/CIFAR-10 dos clases/cat'
imagenes_corruptas = '../Datasets/CIFAR-10 dos clases/cat_c_defocus_blur'

directorio_dataset = pathlib.Path(directorio_dataset)
imagenes_originales = pathlib.Path(imagenes_originales)
imagenes_corruptas = pathlib.Path(imagenes_corruptas)


# Images dimentions
img_height = 32
img_width = 32
channels = 3
batch_size = 32


# Transformaciones para preprocesamiento de datos
transform = transforms.Compose([
    transforms.Resize((img_height, img_width)),  # Ajustar tamaño a 32x32
    transforms.ToTensor(),  # Convertir a tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalizar
])

class CustomImageDataset(Dataset):
    def __init__(self, directory, transform=None):
        self.directory = pathlib.Path(directory)
        self.transform = transform
        self.images = list(self.directory.glob('*.png'))

    def __len__(self):  
        return len(self.images)  

    def __getitem__(self, idx):  
        img_path = self.images[idx]  
        image = Image.open(img_path).convert('RGB')  
        if self.transform:  
            image = self.transform(image)  
        return image


# Crear datasets personalizados
dataset_completo = datasets.ImageFolder(root=directorio_dataset, transform=transform)
dataset_originales = CustomImageDataset(imagenes_originales, transform=transform)
dataset_corruptas = CustomImageDataset(imagenes_corruptas, transform=transform)

# Calcular totales
total_dataset = len(dataset_originales) + len(dataset_corruptas)
total_originales = len(dataset_originales)
total_corruptas = len(dataset_corruptas)


# Crear el DataLoader para iterar sobre los datos en lotes
# DataLoader para cargar los datasets
dataloader_completo = DataLoader(dataset_completo, batch_size=batch_size, shuffle=False, num_workers=0)
dataloader_originales = DataLoader(dataset_originales, batch_size=batch_size, shuffle=False)
dataloader_corruptas = DataLoader(dataset_corruptas, batch_size=batch_size, shuffle=False)

# # DataLoader para cargar los datasets  
# dataloader_completo = DataLoader(dataset_completo, batch_size=batch_size, shuffle=False, num_workers=0)  

print(f"Total imágenes dataset: {total_dataset}")  
print(f"Total imágenes originales: {total_originales}")  
print(f"Total imágenes corruptas: {total_corruptas}")

class_names = dataset_completo.classes
num_classes = len(class_names)

print(f"Número de clases: {num_classes}")
print(f"Los nombres de las clases son: {class_names}")

Total imágenes dataset: 10000
Total imágenes originales: 5000
Total imágenes corruptas: 5000
Número de clases: 2
Los nombres de las clases son: ['cat', 'cat_c_defocus_blur']


In [16]:
def get_transform(channels):   # Aqui define los canales de entrada que se aplicaran al modelo
    if channels == 3:
        # Definir transformaciones para el conjunto de datos en escala a color
        transform = transforms.Compose([
            transforms.Resize((img_height, img_width)),  # Ajustar tamaño
            transforms.ToTensor(),  # Convertir a tensor
            transforms.Normalize((0.5,), (0.5,))  # Normalizar
        ])
    elif channels == 1:
        # Definir transformaciones para el conjunto de datos en escala de grises
        transform = transforms.Compose([
            transforms.Grayscale(),  # Convertir a escala de grises
            transforms.Resize((img_height, img_width)),  # Ajustar tamaño
            transforms.ToTensor(),  # Convertir a tensor
            transforms.Normalize((0.5,), (0.5,))  # Normalizar
        ])
    else: 
        print("El valor elegido no está dentro del rango, por favor elija nuevamente")
        return None  # Devolver None en caso de elección inválida
    return transform

In [17]:
# Obtener las transformaciones según la elección  
transform = get_transform(channels)  

# Crear el dataset completo con las transformaciones aplicadas  
if transform is not None:  
    train_dataset = datasets.ImageFolder(root=directorio_dataset, transform=transform)  # En train dataset se toma el dataset completo para entrenarlo
else:  
    print("No se puede crear el dataset debido a una elección inválida.")  


# Crear el DataLoader para iterar sobre el dataset completo  
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0) 

In [18]:

# Definir el modelo de clasificación multiclase  
class MulticlassCNN(nn.Module):  
    def __init__(self, num_classes=2):  
        super(MulticlassCNN, self).__init__()  
        self.conv1 = nn.Conv2d(channels, 16, kernel_size=3, stride=1, padding=1)  
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)  
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)  
        self.fc1 = nn.Linear(32 * 8 * 8, 128)  
        self.fc2 = nn.Linear(128, num_classes)  
        self.relu = nn.ReLU()  

    def forward(self, x):  
        x = self.pool(self.relu(self.conv1(x)))  
        x = self.pool(self.relu(self.conv2(x)))  
        x = x.view(-1, 32 * 8 * 8)  
        x = self.relu(self.fc1(x))  
        x = self.fc2(x)  
        return x  

# Instanciar el modelo  
model = MulticlassCNN(num_classes=num_classes).to(device)  

# Definir la función de pérdida y el optimizador  
criterion = nn.CrossEntropyLoss()  
optimizer = optim.Adam(model.parameters(), lr=0.001)  

# Ejemplo de bucle de entrenamiento  
num_epochs = 10  
for epoch in range(num_epochs):  
    model.train()  # Asegurarse de que el modelo está en modo de entrenamiento  

    running_loss = 0.0  
    correct = 0  
    total = 0  

    for images, labels in train_loader:  
        images, labels = images.to(device), labels.to(device)  

        # Adelante  
        outputs = model(images)  
        loss = criterion(outputs, labels)  

        # Atrás y optimización  
        optimizer.zero_grad()  
        loss.backward()  
        optimizer.step()  

        # Actualizar estadísticas  
        running_loss += loss.item()  

        # Calcular precisión  
        _, predicted = torch.max(outputs, 1)  
        total += labels.size(0)  
        correct += (predicted == labels).sum().item()  

    epoch_loss = running_loss / len(train_loader)  
    accuracy = 100 * correct / total  

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.2f}%")  

print("Entrenamiento completado")

Epoch [1/10], Loss: 0.5153
Epoch [2/10], Loss: 0.2910
Epoch [3/10], Loss: 0.4930
Epoch [4/10], Loss: 0.4904
Epoch [5/10], Loss: 0.2689
Epoch [6/10], Loss: 0.0833
Epoch [7/10], Loss: 0.1043
Epoch [8/10], Loss: 0.1485
Epoch [9/10], Loss: 0.0736
Epoch [10/10], Loss: 0.0099
Entrenamiento completado
